In [4]:
pip install hazm

In [5]:
import pandas as pd
import tensorflow as tf
from hazm import *
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score

In [6]:
train_df = pd.read_csv('/content/drive/MyDrive/AI_4022/data/train_data.csv')
print(train_df.head())

                                               title  \
0  روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...   
1  انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...   
2  ‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...   
3  واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...   
4  مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...   

                                         description     tags  
0  گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...  استانها  
1  گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...  استانها  
2  گروه استان‌ها ــ با اعلام شیوع ویروس کرونا در ...  استانها  
3  یک متخصص بیوتکنولوژی پزشکی با بیان اینکه مرگ‌و...  اجتماعی  
4  رئیس اتحادیه فروشگاه‌های زنجیره‌ای با اشاره به...  اقتصادی  


In [7]:
print(train_df.describe())

                                                    title  \
count                                               12457   
unique                                              12449   
top     عراقچی برای شرکت در نشست کمیسیون مشترک برجام و...   
freq                                                    2   

                                              description     tags  
count                                               12457    12455  
unique                                              12324      100  
top     گروه استان‌ها ــ این روزها همه‌جا صحبت از جولا...  بایگانی  
freq                                                    3     1944  


In [8]:
train_df.dropna(inplace=True)
print(train_df.describe())

                                                   title  \
count                                              12455   
unique                                             12447   
top     نخست وزیر پیشین فرانسه به پنج سال زندان محکوم شد   
freq                                                   2   

                                              description     tags  
count                                               12455    12455  
unique                                              12322      100  
top     رییس‌جمهور گفت: امسال سال سختی است؛ هم با ویرو...  بایگانی  
freq                                                    3     1944  


In [9]:
print(train_df['tags'].unique())
print(len(train_df['tags'].unique()))

['استانها' 'اجتماعی' 'اقتصادی' 'سیاسی' 'فرهنگی' 'بین الملل' 'ورزشی'
 'رسانه ها' 'بازار' 'الشرق الأوسط' 'دانش و محیط زیست' 'جهان' 'ایران'
 'NRS-Import' 'فرهنگ و هنر' 'آلمان' 'دیدگاه' 'دوره\u200cهای زبان آلمانی'
 'ورزش' 'اقتصاد' 'گوناگون' 'شبکه\u200cهای اجتماعی' 'دویچه وله'
 'Community D' 'آلمانی پیش\u200cرفته' 'ویدئو > گزارش' 'استان ها > مرکزی'
 'استان ها > قم' 'علمی و دانشگاهی > صنفی، فرهنگی'
 'اقتصادی > عمران و اشتغال' 'اقتصادی > تولید و تجارت' 'اجتماعی > سلامت'
 'اجتماعی > آموزش و پرورش' 'سیاسی > دولت'
 'فرهنگی و هنری > گردشگری و میراث' 'اقتصادی > اقتصاد کلان'
 'ورزشی > فوتبال، فوتسال' 'سیاسی > سیاست خارجی' 'اجتماعی > حوادث، انتظامی'
 'سیاسی > مجلس' 'اجتماعی > محیط زیست' 'سیاسی > اندیشه امام و رهبری'
 'اجتماعی > خانواده' 'سیاسی > حقوقی و قضایی' 'سیاسی > سیاست داخلی'
 'استان ها > لرستان' 'علمی و دانشگاهی > علم و فناوری ایران'
 'استان ها > گیلان' 'اجتماعی > جامعه، شهری' 'سیاسی > دفاعی - امنيتی'
 'علمی و دانشگاهی > پژوهش' 'گرافیک > اینفوگرافیک' 'استان ها > خوزستان'
 'اقتصادی > انرژی' 'ف

In [10]:
# limit 'tags' column value
main_tags = ['اجتماعی', 'اقتصاد', 'استان', 'بین الملل', 'سیاسی', 'ورزش']

for tag in main_tags:
  train_df.loc[train_df['tags'].str.contains(tag), 'tags'] = tag

train_df.loc[train_df['tags'].str.contains('علم|فرهنگ'), 'tags'] = 'اجتماعی'
train_df.loc[train_df['tags'].str.contains('بازار'), 'tags'] = 'اقتصاد'
train_df.loc[train_df['tags'].str.contains('ايران|آلمان|جهان|[a-z, A-z]'), 'tags'] = 'بین الملل'
train_df.loc[train_df['tags'].str.contains('ایران'), 'tags'] = 'سیاسی'

print(train_df['tags'].unique())
print(len(train_df['tags'].unique()))
print(train_df.describe())
print(train_df.groupby(['tags'], as_index=False).count())

['استان' 'اجتماعی' 'اقتصاد' 'سیاسی' 'بین الملل' 'ورزش' 'دیدگاه' 'گوناگون'
 'بایگانی' 'خبرخوان']
10
                                                   title  \
count                                              12455   
unique                                             12447   
top     نخست وزیر پیشین فرانسه به پنج سال زندان محکوم شد   
freq                                                   2   

                                              description   tags  
count                                               12455  12455  
unique                                              12322     10  
top     رییس‌جمهور گفت: امسال سال سختی است؛ هم با ویرو...  سیاسی  
freq                                                    3   3905  
        tags  title  description
0    اجتماعی   1619         1619
1      استان    585          585
2     اقتصاد    916          916
3    بایگانی   1944         1944
4  بین الملل   3284         3284
5    خبرخوان     20           20
6     دیدگاه     31           31
7

In [11]:
print(train_df[train_df['tags'] == 'بایگانی'])

                                                   title  \
9763   شرکت هواپیمایی ترکیه پروازهایش به چهار شهر ایر...   
9764   ادامه کشمکش لفظی بین وزارت بهداشت و نماینده مج...   
9766   شمخانی نماینده قم را به دلیل سخنانش درباره کرو...   
9767   وزیر دادگستری مامور پیگیری اتهامات نیلوفر بیان...   
9768   دادگاه جدید پرونده ۱۶ هزار میلیارد تومانی گروه...   
...                                                  ...   
12450   جدایی‌طلبان یمن از طرح خودمختاری عقب‌نشینی کردند   
12451  «بازداشت ۴۰ نفر» در ارتباط با آتش‌سوزی جنگل‌ها...   
12454  روسیه: ابراز نگرانی لاوروف و ظریف درباره حضور ...   
12455                     وزیر خارجه ایران وارد بیروت شد   
12456         هوشنگ ظریف، استاد موسیقی سنتی ایران درگذشت   

                                             description     tags  
9763   در پی شیوع ویروس کرونا در ایران شرکت هواپیمایی...  بایگانی  
9764   کشمکش لفظی بین معاون وزارت بهداشت و نماینده قم...  بایگانی  
9766   علی شمخانی، دبیر شورای عالی امنیت ملی ایران، خ...  بایگانی  
9767   

In [12]:
# ignore datas which are not in specified categories
train_df = train_df[train_df['tags'].isin(main_tags)]

print(train_df['tags'].unique())
print(len(train_df['tags'].unique()))
print(train_df.describe())
print(train_df.head())
print(train_df.groupby(['tags'], as_index=False).count())

['استان' 'اجتماعی' 'اقتصاد' 'سیاسی' 'بین الملل' 'ورزش']
6
                                                    title description   tags
count                                               10458       10458  10458
unique                                              10452       10326      6
top     دستور رئیس جمهور برای ساماندهی و رسیدگی سریع ب...        . \n  سیاسی
freq                                                    2           3   3905
                                               title  \
0  روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...   
1  انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...   
2  ‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...   
3  واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...   
4  مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...   

                                         description     tags  
0  گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...    استان  
1  گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...    استان  
2  گروه استان‌ها ــ با اعلام

In [13]:
# map each cat to a number
categories = dict([(x, i) for i, x in enumerate(main_tags)])
print(categories)

{'اجتماعی': 0, 'اقتصاد': 1, 'استان': 2, 'بین الملل': 3, 'سیاسی': 4, 'ورزش': 5}


In [14]:
# make categories numerical
train_df['tags'] = train_df['tags'].replace(categories)
print(train_df['tags'].unique())
print(train_df.head())

[2 0 1 4 3 5]
                                               title  \
0  روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...   
1  انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...   
2  ‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...   
3  واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...   
4  مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...   

                                         description  tags  
0  گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...     2  
1  گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...     2  
2  گروه استان‌ها ــ با اعلام شیوع ویروس کرونا در ...     2  
3  یک متخصص بیوتکنولوژی پزشکی با بیان اینکه مرگ‌و...     0  
4  رئیس اتحادیه فروشگاه‌های زنجیره‌ای با اشاره به...     1  


<ipython-input-14-d9704b7357ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['tags'] = train_df['tags'].replace(categories)


In [15]:
normalizer = Normalizer()
stemmer = Stemmer()
stop_words = stopwords_list()
lemmatizer = Lemmatizer()

def preprocess(text):
    # remove links, signs, english alphabet, and numbers
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'[\:|\-|\&|\;|\#|\.|\+|\/|0-9|quot]','',text)

    # Normalize the text
    text = normalizer.correct_spacing(text)
    text = normalizer.remove_specials_chars(text)
    text = normalizer.normalize(text)

    # Tokenize the text
    words = word_tokenize(text)
    # with hazm
    # tokenizer = WordTokenizer()
    # words = tokenizer.tokenize(text)

    # Remove stopwords and stem the words
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return ' '.join(words)

train_df['title'] = train_df['title'].apply(preprocess)
train_df['description'] = train_df['description'].apply(preprocess)
print(train_df.head())

                                               title  \
0  روایت تحصیل زیرسقف لرز سرما سوز ؛ دانش‌آموز که...   
1  انقلاب سرشناس بحرین گفت‌وگو تسن مرد منطقه پیشر...   
2         زندگ مرد شهر ق پیشگیر اولو مرد قرار تصاویر   
3   واکن پروفسور کرم دروغ‌پراکن رسانه القا ترس کرونا   
4  مرد نگر تأمین کالا موردنیاز باشید#باش میز تولی...   

                                         description  tags  
0  گروه استان دانش‌آموز مدرسه روستا کهنانیک بلوچس...     2  
1  گروه استان انقلاب سرشناس بحرین مرد منطقه پیشرف...     2  
2  گروه استان اعلا شیوع ویروس کرونا شهر ق زندگ عا...     2  
3  متخصص بیوتکنولوژ پزشک مرگ‌و‌میر حاصل کرونا به‌...     0  
4  رئیس اتحادیه فروشگاه زنجیره اشاره خرید اندازه ...     1  


In [16]:
print(train_df)

                                                   title  \
0      روایت تحصیل زیرسقف لرز سرما سوز ؛ دانش‌آموز که...   
1      انقلاب سرشناس بحرین گفت‌وگو تسن مرد منطقه پیشر...   
2             زندگ مرد شهر ق پیشگیر اولو مرد قرار تصاویر   
3       واکن پروفسور کرم دروغ‌پراکن رسانه القا ترس کرونا   
4      مرد نگر تأمین کالا موردنیاز باشید#باش میز تولی...   
...                                                  ...   
12422          قاض آمریکا فر حذف پیام‌رس چین « چ » متوقف   
12441  واکن فرماندار کرمانشاه ویدئو آزار دخ خاطر حجاب...   
12447        دادستان خراس رضو دوچرخه‌سوار زن ممنوع نکرده   
12452                  علیرضا زاکان رئیس مرکز پژوهش مجلس   
12453  قوه قضائیه تقاضا فرجا پرونده روح‌الله ز دیو رف...   

                                             description  tags  
0      گروه استان دانش‌آموز مدرسه روستا کهنانیک بلوچس...     2  
1      گروه استان انقلاب سرشناس بحرین مرد منطقه پیشرف...     2  
2      گروه استان اعلا شیوع ویروس کرونا شهر ق زندگ عا...     2  
3      متخصص بیوتکن

In [17]:
# normalizer = Normalizer()
# def normal(text):
#   text = normalizer.normalize(text)
#   return text

# train_df['title'] = train_df['title'].apply(normal)
# train_df['description'] = train_df['description'].apply(normal)
# train_df.head()

In [18]:
# stop_words = stopwords_list()
# def remove_stopwords(text):
#   # words = text.split(' ')
#   words = word_tokenize(text)
#   words = [word for word in words if word not in stop_words]
#   return ' '.join(words)

# train_df['title'] = train_df['title'].apply(remove_stopwords)
# train_df['description'] = train_df['description'].apply(remove_stopwords)
# train_df.head()

In [19]:
# stemmer = Stemmer()
# def stem(text):
#   # words = text.split(' ')
#   words = word_tokenize(text)
#   words = [stemmer.stem(word) for word in words]
#   return ' '.join(words)

# train_df['title'] = train_df['title'].apply(stem)
# train_df['description'] = train_df['description'].apply(stem)
# train_df.head()

In [20]:
# lemmatizer = Lemmatizer()
# def lemma(text):
#   # words = text.split(' ')
#   words = word_tokenize(text)
#   words = [lemmatizer.lemmatize(word) for word in words]
#   return ' '.join(words)

# train_df['title'] = train_df['title'].apply(lemma)
# train_df['description'] = train_df['description'].apply(lemma)
# train_df.head()

In [21]:
# title is more important
train_df['text'] = train_df['title'] + ' ' + train_df['title'] + ' ' + train_df['description']
print(train_df.head())

                                               title  \
0  روایت تحصیل زیرسقف لرز سرما سوز ؛ دانش‌آموز که...   
1  انقلاب سرشناس بحرین گفت‌وگو تسن مرد منطقه پیشر...   
2         زندگ مرد شهر ق پیشگیر اولو مرد قرار تصاویر   
3   واکن پروفسور کرم دروغ‌پراکن رسانه القا ترس کرونا   
4  مرد نگر تأمین کالا موردنیاز باشید#باش میز تولی...   

                                         description  tags  \
0  گروه استان دانش‌آموز مدرسه روستا کهنانیک بلوچس...     2   
1  گروه استان انقلاب سرشناس بحرین مرد منطقه پیشرف...     2   
2  گروه استان اعلا شیوع ویروس کرونا شهر ق زندگ عا...     2   
3  متخصص بیوتکنولوژ پزشک مرگ‌و‌میر حاصل کرونا به‌...     0   
4  رئیس اتحادیه فروشگاه زنجیره اشاره خرید اندازه ...     1   

                                                text  
0  روایت تحصیل زیرسقف لرز سرما سوز ؛ دانش‌آموز که...  
1  انقلاب سرشناس بحرین گفت‌وگو تسن مرد منطقه پیشر...  
2  زندگ مرد شهر ق پیشگیر اولو مرد قرار تصاویر زند...  
3  واکن پروفسور کرم دروغ‌پراکن رسانه القا ترس کرو...  
4  مرد نگر تأمی

In [22]:
# vectorize the data to be able to work with it
vectorizer = TfidfVectorizer()
# vectorizer = CountVectorizer()
vectorizer.fit(train_df['text'])
X = vectorizer.transform(train_df['text'])

In [23]:
# le = LabelEncoder()
# y = le.fit_transform(data['tags'])
y = train_df['tags']
# le.inverse_transform([0, 1, 2, 3, 4, 5])

In [24]:
# split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# Test different classifiers to find the best
def run_model(model_name):
  model=''
  if model_name == 'Logistic Regression':
    model = LogisticRegression()
  elif model_name == 'Random Forest':
    model = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0)
  elif model_name == 'Multinomial Naive Bayes':
    model = MultinomialNB(alpha=1.0,fit_prior=True)
  elif model_name == 'Support Vector Classifer':
    model = SVC(random_state=123)
  elif model_name == 'Decision Tree Classifier':
    model = DecisionTreeClassifier()
  elif model_name == 'K Nearest Neighbour':
    model = KNeighborsClassifier(n_neighbors=5 , metric= 'euclidean' , p = 4)
  elif model_name == 'Gaussian Naive Bayes':
    model = GaussianNB()

  oneVsRest = OneVsRestClassifier(model)
  oneVsRest.fit(X_train, y_train)
  y_pred = oneVsRest.predict(X_test)

  # Performance metrics
  accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

  # Get precision, recall, f1 scores
  precision, recall, f1score, support = score(y_test, y_pred, average='micro')

  print(f'Test Accuracy Score of Basic {model_name}: % {accuracy}')
  print(f'Precision : {precision}')
  print(f'Recall : {recall}')
  print(f'F1-score : {f1score}')

  # Add performance parameters to list
  perform_list.append(dict([('Model', model_name),
                            ('Test Accuracy', round(accuracy, 2)),
                            ('Precision', round(precision, 2)),
                            ('Recall', round(recall, 2)),
                            ('F1', round(f1score, 2))]))

In [26]:
# Create list of model and accuracy dicts
perform_list = []

In [27]:
run_model('Logistic Regression')

Test Accuracy Score of Basic Logistic Regression: % 81.98
Precision : 0.8197896749521989
Recall : 0.8197896749521989
F1-score : 0.8197896749521989


In [28]:
run_model('Random Forest')

Test Accuracy Score of Basic Random Forest: % 84.32
Precision : 0.8432122370936902
Recall : 0.8432122370936902
F1-score : 0.8432122370936903


In [29]:
run_model('Multinomial Naive Bayes')

Test Accuracy Score of Basic Multinomial Naive Bayes: % 64.24
Precision : 0.6424474187380497
Recall : 0.6424474187380497
F1-score : 0.6424474187380497


In [30]:
run_model('Support Vector Classifer')

Test Accuracy Score of Basic Support Vector Classifer: % 83.51
Precision : 0.8350860420650096
Recall : 0.8350860420650096
F1-score : 0.8350860420650096


In [31]:
run_model('Decision Tree Classifier')

Test Accuracy Score of Basic Decision Tree Classifier: % 69.89
Precision : 0.6988527724665392
Recall : 0.6988527724665392
F1-score : 0.6988527724665392


In [32]:
# takes a lot of time
# run_model('K Nearest Neighbour')

In [33]:
# run_model('Gaussian Naive Bayes')

In [34]:
# Compare the results of different models
model_performance = pd.DataFrame(data=perform_list)
model_performance = model_performance[['Model', 'Test Accuracy', 'Precision', 'Recall', 'F1']]
model_performance

,Model,Test Accuracy,Precision,Recall,F1
0,Logistic Regression,81.98,0.82,0.82,0.82
1,Random Forest,84.32,0.84,0.84,0.84
2,Multinomial Naive Bayes,64.24,0.64,0.64,0.64
3,Support Vector Classifer,83.51,0.84,0.84,0.84
4,Decision Tree Classifier,69.89,0.70,0.70,0.70


In [35]:
# defining parameter range for SVC
# 10, 0.1
param_grid = {'C': [1, 10, 100],
              'gamma': [1, 0.1, 0.01],
              'kernel': ['rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.834 total time= 1.1min
[CV 2/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.827 total time= 1.1min
[CV 3/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.831 total time= 1.0min
[CV 4/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.834 total time= 1.0min
[CV 5/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.829 total time= 1.1min
[CV 1/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.775 total time=  50.3s
[CV 2/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.764 total time=  50.2s
[CV 3/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.778 total time=  49.7s
[CV 4/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.759 total time=  50.7s
[CV 5/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.759 total time=  49.8s
[CV 1/5] END .......C=1, gamma=0.01, kernel=rbf;, score=0.539 total time= 1.1min
[CV 2/5] END .......C=1, gamma=0.01, kernel=rbf;,

In [36]:
print(grid.best_estimator_)

SVC(C=10, gamma=0.1)


In [37]:
classifier = SVC(random_state=123, C=10, gamma=0.1)
classifier.fit(X_train, y_train)
# print(le.inverse_transform([y_test[4]]))
# le.inverse_transform([svc.predict(X_test[4])])

SVC(C=10, gamma=0.1, random_state=123)

In [38]:
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.76      0.76       308
           1       0.79      0.80      0.80       174
           2       0.78      0.66      0.72       116
           3       0.88      0.88      0.88       653
           4       0.86      0.88      0.87       809
           5       0.96      0.75      0.84        32

    accuracy                           0.84      2092
   macro avg       0.84      0.79      0.81      2092
weighted avg       0.84      0.84      0.84      2092



In [39]:
test_df = pd.read_csv('/content/drive/MyDrive/AI_4022/data/test_data.csv')
print(test_df.head())

                                               title  \
0  درگیر شایعات نشوید؛ نتیجه آزمایش افراد مشکوک ب...   
1  تخریب منازل «علی‌آباد» خرم‌آباد در پی بارش بار...   
2  ‌اختصاصی| قرنطینه آیت‌الله شبیری‌زنجانی صحت ند...   
3  احتمال طغیان رودخانه‌های غرب کشور/ کاهش شدت با...   
4  تحویل طومار مردم تهران برای پیگرد عاملان ترور ...   

                                         description  
0  گروه استان‌ها- رئیس دانشگاه علوم پزشکی استان ا...  
1  گروه استان‌ها ـبارش‌های سیل‌آسا که از روز گذشت...  
2  گروه استان‌ها ـ مسئول دفتر آیت الله شبیری زنجا...  
3  رئیس سازمان هواشناسی از احتمال طغیان اغلب رودخ...  
4  طومار امضا شده توسط هزاران تن از مردم تهران در...  


In [40]:
# preprocess test data
test_df['title'] = test_df['title'].apply(preprocess)
test_df['description'] = test_df['description'].apply(preprocess)
test_df['text'] = test_df['title'] + ' ' + test_df['title'] + ' ' + test_df['description']
X_test_df = vectorizer.transform(test_df['text'])
print(test_df)

                                                 title  \
0    درگیر شایع شد#شو ؛ نتیجه آزما مشکوک کرونا ایلا...   
1         تخریب منازل « علی‌آباد » خرم‌آباد روا تصاویر   
2               اختصاص قرنطینه آیت‌الله شبیری‌زنجان صح   
3    احتمال طغ رودخانه غرب کشور کاه بارش تهر ظهر امروز   
4    تحویل طومار مرد تهر پیگرد عامل ترور شهید سلیما...   
..                                                 ...   
995      جو بایدن بسیج وکلا دعوا حقوق انتخاب آماد#آمای   
996            کاه ۹۶ درصد مسافر بین‌الملل صنع هوا ایر   
997    امار متحده عرب سال ۲۰۱۴ فضاپیما ماه فرستاد#فرست   
998     کیو خسرو به‌دلیل ملاحظ امنیت حادثه نطنز ز اعلا   
999       گروه ناشناس مسئول « حمله تاسیس نطنز » برعهده   

                                           description  \
0    گروه استان رئیس دانشگاه علو پزشک اس ایلا منف ن...   
1    گروه استان بارش سیل‌آسا روز گذشته لرس آغاز ، م...   
2    گروه استان مسئول دف آیت‌الله شبیر زنجان آیت‌ال...   
3    رئیس هواشناس احتمال طغ رودخانه غرب کشور خبر گف...   
4    طومار ام

In [41]:
print(y_pred.shape)
# use the model with best accuracy
y_pred_test_df = classifier.predict(X_test_df)

(2092,)


In [42]:
test_df['prediction'] = y_pred_test_df
# df = pd.DataFrame({'prediction':y_pred_test_df})
# print(df.head())

# save the result dataframe as a CSV file
submission_df = test_df[['prediction']]
# submission_df = df[['prediction']]
submission_df.to_csv('submission.csv', index=False)